In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from modelisation.functions import load_file, save_files
from db.database_mysql import engine

import xgboost 
from sklearn import set_config
set_config(transform_output="pandas")
pd.set_option('display.max_columns', 500)

In [8]:
def set_score(val, q):
  s = q.loc[val >= q.total].tail(1).index
  s = s[0] if s.shape[0] > 0 else 0.25
  return np.exp(s)

def calculate(df, col_name, use_col_spectator = 'total'):
  q = df.quantile([0.01, .1, .25, .5, .75, .9, .95], numeric_only=True)

  score = df.copy()
  score[f'{col_name}_combined_score'] = df[use_col_spectator].apply(set_score, q=q)

  save_files(
          score[[col_name, f"{col_name}_combined_score"]],
          f"{col_name}_scores",
      )

# DIRECTOR

In [9]:
df = pd.read_sql_query('''SELECT im.director as director, sum(jp.first_week) as week, sum(jp.total_spectator) as total, count(im.id) as nb_film
FROM films_imdb as im
LEFT JOIN films_jp jp ON jp.id = im.id_jp 
where im.id_jp is not null and im.date = jp.date
group by im.director
order by total desc''', engine)

calculate(df, 'director')

# ACTOR

In [10]:
df = pd.read_sql_query('''SELECT actor, sum(jp.first_week) as week, sum(jp.total_spectator) as total, count(im.id) as nb_film
FROM films_imdb as im
join
   JSON_TABLE(
     im.casting,
     "$[*]"
     COLUMNS(
       actor VARCHAR(255) PATH "$"
     )
   ) as aa
LEFT JOIN films_jp jp ON jp.id = im.id_jp 
where im.id_jp is not null and im.date = jp.date

group by actor
order by total desc''', engine)

calculate(df, 'actor')

# DISTRIBUTOR

In [11]:
df = pd.read_sql_query('''SELECT dist, sum(jp.first_week) as week, sum(jp.total_spectator) as total, count(im.id) as nb_film
FROM films_imdb as im
join
   JSON_TABLE(
     im.distributor,
     "$[*]"
     COLUMNS(
       dist VARCHAR(255) PATH "$"
     )
   ) as aa
LEFT JOIN films_jp jp ON jp.id = im.id_jp 
where im.id_jp is not null and im.date = jp.date

group by dist
order by total desc''', engine)

df.rename(columns={"dist": "distributor"}, inplace=True)
calculate(df, 'distributor')

In [13]:
q = load_file('director_scores')

q

,director,director_combined_score
0,"""dany boon""",2.585710
1,"""james cameron""",2.585710
2,"""steven spielberg""",2.585710
3,"""luc besson""",2.585710
4,"""peter jackson""",2.585710
...,...,...
1599,"""frederic balekdjian""",1.284025
1600,"""micha wald""",1.284025
1601,"""germinal alvarez""",1.284025
1602,"""callie khouri""",1.284025


# GENRE

In [14]:
df = pd.read_sql_query('''SELECT gen, sum(jp.first_week) as week, sum(jp.total_spectator) as total, count(im.id) as nb_film
FROM films_imdb as im
join
   JSON_TABLE(
     im.genre,
     "$[*]"
     COLUMNS(
       gen VARCHAR(255) PATH "$"
     )
   ) as aa
LEFT JOIN films_jp jp ON jp.id = im.id_jp 
where im.id_jp is not null and im.date = jp.date

group by gen
order by total desc''', engine)

df.rename(columns={"gen": "genre"}, inplace=True)
calculate(df, 'genre')

In [16]:
q = load_file('genre_scores')
q

,genre,genre_combined_score
0,comedie,2.585710
1,aventure,2.585710
2,drame,2.459603
3,action,2.117000
4,animation,2.117000
5,policier,2.117000
6,romantique,1.648721
7,thriller,1.648721
8,fantastique,1.648721
9,famille,1.648721
